In [2]:
import keras
from keras import layers

import numpy as np
import random
import io

In [3]:
path = "text_example.txt"
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  
text = text.replace("“", " ")
text = text.replace("”", " ")
text = text.replace("(", " ")
text = text.replace(")", " ")
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Corpus length: 114708
Total chars: 48
Number of sequences: 38223


In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [5]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
epochs = 20
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.5, 1.0]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(100):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print("-")

299/299 [==============================] - 14s 47ms/step - loss: 1.1135

Generating text after epoch: 0
...Diversity: 0.5
...Generating with seed: "as trevelyan, up there. and armand treve"
...Generated:  lyan fult to he books who con’s one of the and trevely, y.  i was doing.    that’s mys one on a pasi
-
...Diversity: 1.0
...Generating with seed: "as trevelyan, up there. and armand treve"
...Generated:  lyans aren’t over and day.  the centing.  george thes fuy. his felt achtomation at wanted matter. in
-
299/299 [==============================] - 15s 49ms/step - loss: 1.0894

Generating text after epoch: 1
...Diversity: 0.5
...Generating with seed: "ut you are now. let me introduce myself,"
...Generated:    george looked a registered men founder and then i competing at that was no were not mosely that i 
-
...Diversity: 1.0
...Generating with seed: "ut you are now. let me introduce myself,"
...Generated:   are but was it indiftelly, he ar—  but otherly,  passed i’m the notting. only 